In [ ]:
# define train and validation generators here

## MODEL: ResNet50 Finetuned

In [ ]:
top_model_weights_path = '../bottleneck_fc_model.h5'

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64,64,3))

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights(top_model_weights_path)

In [ ]:
model = Sequential()
model.add(base_model)
model.add(top_model)

for layer in model.layers[:50]:
    layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
log_file_path = 'logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = 'model_weights_resnet/' + dataset_name + '_resnet50_finetune'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
resnet_finetune = model.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size, 
                                 epochs=num_epochs, validation_data=validation_generator, 
                                 validation_steps=nb_validation_samples // batch_size, verbose=1,
                                 callbacks=callbacks)